In [1]:
%pip install python-docx
%pip install elasticsearch

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from docx import Document

def clean_word_remove_footer(input_path, output_path):
    # Mở tài liệu Word
    doc = Document(input_path)
    
    for section in doc.sections:
        # Xóa nội dung trong footer
        for paragraph in section.footer.paragraphs:
            paragraph.text = ""  # Làm trống nội dung
        
        # Xóa nội dung trong header (nếu cần)
        for paragraph in section.header.paragraphs:
            paragraph.text = ""  # Làm trống nội dung
    
    # Lưu tài liệu đã làm sạch
    doc.save(output_path)
    print(f"Footer và Header đã được làm trống. Tệp mới lưu tại: {output_path}")

# Sử dụng
clean_word_remove_footer("QUỐC HỘI.docx", "52.2014.QH13_clean.docx")


Footer và Header đã được làm trống. Tệp mới lưu tại: 52.2014.QH13_clean.docx


In [3]:
from docx import Document

def clean_word_based_on_keywords(input_path, output_path, keywords_to_remove_start, keywords_to_remove_end):
    """
    Xóa các phần đầu và cuối tài liệu dựa trên từ khóa xác định.
    
    input_path: Đường dẫn file gốc (.docx)
    output_path: Đường dẫn file kết quả (.docx)
    keywords_to_remove_start: Danh sách từ khóa để xác định phần đầu không liên quan.
    keywords_to_remove_end: Danh sách từ khóa để xác định phần cuối không liên quan.
    """
    doc = Document(input_path)
    paragraphs = doc.paragraphs
    new_doc = Document()

    # Tìm và loại bỏ các phần đầu không liên quan
    start_index = 0
    for i, paragraph in enumerate(paragraphs):
        if any(keyword in paragraph.text for keyword in keywords_to_remove_start):
            start_index = i + 1  # Bắt đầu từ đoạn sau phần đầu không liên quan
        else:
            break

    # Tìm và loại bỏ các phần cuối không liên quan
    end_index = len(paragraphs)
    for i, paragraph in enumerate(reversed(paragraphs)):
        if any(keyword in paragraph.text for keyword in keywords_to_remove_end):
            end_index = len(paragraphs) - i - 1  # Dừng lại ở đoạn trước phần cuối không liên quan
        else:
            break

    # Thêm các đoạn văn còn lại vào tài liệu mới
    for i, paragraph in enumerate(paragraphs[start_index:end_index]):
        new_doc.add_paragraph(paragraph.text)

    # Lưu tài liệu đã làm sạch
    new_doc.save(output_path)
    print(f"Đã làm sạch tài liệu. Tệp lưu tại: {output_path}")

# Sử dụng:
keywords_to_remove_start = ["Cộng hòa xã hội chủ nghĩa Việt Nam", "Quốc hội", "Chủ tịch"]
keywords_to_remove_end = ["Chữ ký", "Phê duyệt", "Chủ tịch"]

clean_word_based_on_keywords(
    "QUỐC HỘI.docx",
    "52.2014.QH13_clean.docx",
    keywords_to_remove_start=keywords_to_remove_start,
    keywords_to_remove_end=keywords_to_remove_end
)

Đã làm sạch tài liệu. Tệp lưu tại: 52.2014.QH13_clean.docx


In [4]:
import re
from docx import Document
from elasticsearch import Elasticsearch

# Kết nối với Elasticsearch (sử dụng localhost hoặc dịch vụ Elasticsearch)
es = Elasticsearch("http://localhost:9200")  # URL của Elasticsearch


def chunking_law_text(input_path):
    doc = Document(input_path)
    paragraphs = doc.paragraphs

    # Các từ khóa nhận diện chương, điều, và phần điều
    chapters = []
    current_chapter = None
    current_article = None

    for paragraph in paragraphs:
        text = paragraph.text.strip()

        if not text:  # Bỏ qua các đoạn trống
            continue

        # Nhận diện chương: Ví dụ "Chương I", "Chương 1"
        if re.match(r'^\s*CHƯƠNG\s+[IVXLCDM\d]+(\.\d+)*', text):
            current_chapter = text
            chapters.append({'chapter': current_chapter, 'articles': []})
        
        # Nhận diện điều: Ví dụ "Điều 1", "Điều 2"
        elif re.match(r'^\s*Điều\s+\d+', text):
            current_article = text
            if current_chapter:  # Nếu đã có chương, thêm vào danh sách
                chapters[-1]['articles'].append({'article': current_article, 'clauses': []})

        # Nhận diện khoản (dấu số theo sau là dấu chấm): Ví dụ "1.", "2."
        elif re.match(r'^\s*\d+\.', text):  # Ví dụ: "1.", "2."
            if current_article:  # Nếu đã có điều, thêm vào danh sách phần điều
                chapters[-1]['articles'][-1]['clauses'].append(text)

    return chapters

def save_to_elasticsearch(chunks):
    for chapter in chunks:
        for article in chapter['articles']:
            for clause in article['clauses']:
                doc = {
                    "clause_text": clause,
                    "metadata": {
                        "chapter": chapter['chapter'],
                        "article": article['article']
                    }
                }
                # Chỉ định chỉ mục và lưu tài liệu vào Elasticsearch
                es.index(index="law_chunks", document=doc)

# Sử dụng
chunks = chunking_law_text("52.2014.QH13_clean.docx")

# Lưu kết quả vào Elasticsearch
save_to_elasticsearch(chunks)

ConnectionError: Connection error caused by: ConnectionError(Connection error caused by: ProtocolError(('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))))